In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner

In [2]:
folder = os.path.join('..', 'outputs', 'saved_run_20231122t153648')

In [3]:
reader = emcee.backends.HDFBackend(os.path.join(folder, 'backend.hdf'))

In [4]:
chain = reader.get_chain()

In [5]:
print(f"(nstep, nwalker, ndim):  {chain.shape}")

(nstep, nwalker, ndim):  (3, 4, 2)


In [6]:
blobs = reader.get_blobs()

In [7]:
print(blobs['int_time'].shape)

(3, 4, 1, 3)


In [8]:
print(blobs['int_time'])

[[[[nan nan nan]]

  [[nan nan nan]]

  [[nan nan nan]]

  [[nan nan nan]]]


 [[[nan nan nan]]

  [[nan nan nan]]

  [[nan nan nan]]

  [[nan nan nan]]]


 [[[nan nan nan]]

  [[nan nan nan]]

  [[nan nan nan]]

  [[nan nan nan]]]]


In [9]:
# tau = reader.get_autocorr_time(tol=1)

In [10]:
# acc_rate = reader.accepted/chain.shape[0]

In [11]:
# print(acc_rate.mean())

In [12]:
# print(tau.mean())
# print(np.median(tau))

In [13]:
# print(tau)

In [14]:
# plt.plot(chain[:, :, 10])

In [15]:
# flat_chain = reader.get_chain(discard=225, flat=True)
# print(flat_chain.shape)

In [16]:
# labels = ["idrk", "W11", "W12", "W21", "W22", "W31", "W32", "W41", "W42", "W51", "W52", "W61", "W62"
#           , "C3", "C4", "C5", "T3", "T4", "T5"]

In [17]:
# for i in range(flat_chain.shape[1]):
#     fig = corner.corner(flat_chain[:, i:i+1], labels=labels[i])

In [18]:
# fig = corner.corner(flat_chain, labels=labels)